In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_earned = client.open("frb-volley-game-stats").worksheet("player-offense")

# Fetch all records (rows) from the sheet
player_earned_data = player_earned.get_all_records()



In [65]:
# Print the data or convert it into a DataFrame
import pandas as pd
df = pd.DataFrame(player_earned_data)

df["error-pct"] = df["attack-errors"] / df["attack"]
df

,date,match-id,match,set,player,serves,aces,serve-error,points-won,point-scoring-pct,attack,kills,attack-errors,kill-pct,kill-effic,error-pct
0,08/12/2024,141108,frb-gentofte,1,Lasse,2,1,0,1,50.0,1,1,0,100.0,1.0,0.0
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
2,08/12/2024,141108,frb-gentofte,1,Kristian,5,0,2,3,60.0,2,1,0,50.0,0.5,0.0
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,25/01/2025,141139,frb-dtu,4,Vestbjerg,1,0,0,0,0.0,6,3,0,50.0,0.5,0.0
248,25/01/2025,141139,frb-dtu,4,Gustav,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
249,25/01/2025,141139,frb-dtu,4,Mads,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
250,25/01/2025,141139,frb-dtu,4,Martin,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN


In [61]:
summary = df.groupby('player').agg(
    attack_attempts=('attack', 'sum'),
    total_kills=('kills', 'sum'),
    attack_errors=('attack-errors', 'sum')
).reset_index()

summary["kill_pct"] = ((summary["total_kills"]) / summary["attack_attempts"] * 100).round(0)
summary["error_pct"] = ((summary["attack_errors"]) / summary["attack_attempts"] * 100).round(0)
summary["kill_effic"] = ((summary["total_kills"] - summary["attack_errors"]) / summary["attack_attempts"])#.round(3)
summary = summary.dropna()
summary["kill_pct"] = summary["kill_pct"].astype(int)
summary["error_pct"] = summary["error_pct"].astype(int)
summary["kill_effic"] = summary["kill_effic"].round(3)
summary

,player,attack_attempts,total_kills,attack_errors,kill_pct,error_pct,kill_effic
2,Bo,10,4,2,40,20,0.200
3,Boerme,62,27,11,44,18,0.258
5,Gustav,39,10,5,26,13,0.128
6,Kristian,17,6,2,35,12,0.235
7,Lasse,21,10,1,48,5,0.429
8,Mads,1,1,0,100,0,1.000
9,Martin,44,21,7,48,16,0.318
10,Nico,1,0,0,0,0,0.000
11,Nicola,123,56,19,46,15,0.301
12,Soeren,107,37,16,35,15,0.196


In [62]:
import json

# Convert DataFrame to JSON
data = summary.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../player-offense-summary.json", "w") as f:
    json.dump(data, f, indent=4)